In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.74.0 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 5.29.5 which is incompatible.
google-colabsqlviz 0.1.3 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 5.29.5 which is incompatible.


In [1]:
import os
for dir in ['test','train','valid']:
  os.makedirs(f'data/{dir}',exist_ok=True)

In [2]:
!unzip sample.zip -d data/train
!unzip valid.zip -d data/valid/images
!unzip valid_mask.zip -d data/valid/masks

Archive:  sample.zip
   creating: data/train/masks/
  inflating: data/train/masks/10_jpg.rf.25f47c9078615e5a2e5143c6723eef13.png  
  inflating: data/train/masks/1177_jpg.rf.1196c22dff3b71fe3d2190f4bc36232d.png  
  inflating: data/train/masks/1177_jpg.rf.72b70c88c6ee1dbd04f5c4ccd3749721.png  
  inflating: data/train/masks/1212_jpg.rf.66eb74d44adf470bb3f2cc5993e858f1.png  
  inflating: data/train/masks/1267_jpg.rf.946f166f7fc4080f38121174447c3590.png  
  inflating: data/train/masks/1316_jpg.rf.18ca4fe9c8cfee5b968060c34698b737.png  
  inflating: data/train/masks/1325_jpg.rf.a2a0797eafb48c3847b156d780ffe8b6.png  
  inflating: data/train/masks/1368_jpg.rf.13eede3092bcc2ed4c045a3b26d6b8e6.png  
  inflating: data/train/masks/1368_jpg.rf.67ddcd53ba06ba536386b379a5531d45.png  
  inflating: data/train/masks/1385_jpg.rf.1ff0c0da16b5c6a74d187028548b8b3c.png  
  inflating: data/train/masks/1385_jpg.rf.37a4bf38a74a1fd85f264c4854edd774.png  
  inflating: data/train/masks/1385_jpg.rf.b29966bb2a3703129

In [3]:
import cv2
import numpy as np
import os
import random
import shutil

srcdir='data/valid/images'
srcmaskdir='data/valid/masks'


files = [file for file in os.listdir(srcdir) if file.endswith('.jpg')]
random.shuffle(files)
files = files[:300]
images = os.listdir(srcdir)

for file in images:
    if file not in files:
      os.remove(os.path.join(srcdir,file))
      filename = os.path.splitext(file)[0]
      os.remove(os.path.join(srcmaskdir,f'{filename}.png'))

In [59]:
import numpy as np
import cv2


dir = 'data/train'
img_dir = f"{dir}/images"
mask_dir = f"{dir}/masks"
img_size = (224, 224)  # taille d'entrée du modèle
files = [file for file in os.listdir(img_dir) if file.endswith('.jpg')]
images = [cv2.imread(os.path.join(img_dir,file)) for file in files]
masks = [cv2.imread(os.path.join(mask_dir,file.replace('.jpg','.png')), cv2.IMREAD_GRAYSCALE) for file in files]

X, Y = [], []

for img_id,(img,mask) in enumerate(zip(images,masks)):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, img_size)
    X.append(img_resized / 255.0)  # normaliser image
    Y.append(mask)         # masque binaire


In [60]:
# Conversion en numpy
X_train = np.array(X, dtype=np.float32)
Y_train = np.array(Y, dtype=np.uint8)
Y_train = np.expand_dims(Y_train, axis=-1)  # (H, W, 1)

In [58]:
# Conversion en numpy
X_val = np.array(X, dtype=np.float32)
Y_val = np.array(Y, dtype=np.uint8)
Y_val = np.expand_dims(Y_val, axis=-1)  # (H, W, 1)

In [61]:
np.max(Y_val)

np.uint8(1)

Novel

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, Model

def upsample(filters, size):
    """Bloc de déconv simple"""
    return tf.keras.Sequential([
        layers.Conv2DTranspose(filters, size, strides=2, padding="same"),
        layers.BatchNormalization(),
        layers.ReLU()
    ])

def create_segmentation_model(input_shape=(224,224,3)):
    # Charge MobileNetV2 comme backbone
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights="imagenet")

    # On prend certaines couches comme "skip connections"
    layer_names = [
        "block_1_expand_relu",   # 112x112
        "block_3_expand_relu",   # 56x56
        "block_6_expand_relu",   # 28x28
        "block_13_expand_relu",  # 14x14
        "block_16_project",      # 7x7
    ]
    layers_outputs = [base_model.get_layer(name).output for name in layer_names]

    # Modèle encodeur
    encoder = Model(inputs=base_model.input, outputs=layers_outputs)
    encoder.trainable = False  # On freeze au début pour du transfert learning

    # Décodeur (type U-Net)
    inputs = layers.Input(shape=input_shape)
    skips = encoder(inputs)
    x = skips[-1]  # 7x7 feature map

    # Décodage avec skip connections
    up_stack = [
        upsample(512, 3),  # 14x14
        upsample(256, 3),  # 28x28
        upsample(128, 3),  # 56x56
        upsample(64, 3),   # 112x112
    ]

    for up, skip in zip(up_stack, reversed(skips[:-1])):
        x = up(x)
        x = layers.Concatenate()([x, skip])

    # Dernière couche (retour à la taille originale)
    last = layers.Conv2DTranspose(1, 3, strides=2, padding="same", activation="sigmoid")
    outputs = last(x)

    return Model(inputs, outputs)

# Création du modèle
model = create_segmentation_model()
model.summary()


Model: "functional_35"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_31      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_30       │ [(None, 112, 112, │  1,841,984 │ input_layer_31[0… │
│ (Functional)        │ 96), (None, 56,   │            │                   │
│                     │ 56, 144), (None,  │            │                   │
│                     │ 28, 28, 192),     │            │                   │
│                     │ (None, 14, 14,    │            │                   │
│                     │ 576), (None, 7,   │            │                   │
│                     │ 7, 320)]          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_20       │ (None, 14, 14,    │  1,477,120 │ functional_30[0]… │
│ (Sequential)        │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_20      │ (None, 14, 14,    │          0 │ sequential_20[0]… │
│ (Concatenate)       │ 1088)             │            │ functional_30[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_21       │ (None, 28, 28,    │  2,508,032 │ concatenate_20[0… │
│ (Sequential)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_21      │ (None, 28, 28,    │          0 │ sequential_21[0]… │
│ (Concatenate)       │ 448)              │            │ functional_30[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_22       │ (None, 56, 56,    │    516,736 │ concatenate_21[0… │
│ (Sequential)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_22      │ (None, 56, 56,    │          0 │ sequential_22[0]… │
│ (Concatenate)       │ 272)              │            │ functional_30[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_23       │ (None, 112, 112,  │    156,992 │ concatenate_22[0… │
│ (Sequential)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_23      │ (None, 112, 112,  │          0 │ sequential_23[0]… │
│ (Concatenate)       │ 160)              │            │ functional_30[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_29 │ (None, 224, 224,  │      1,441 │ concatenate_23[0… │
│ (Conv2DTranspose)   │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,502,305 (24.80 MB)

 Trainable params: 4,658,401 (17.77 MB)

 Non-trainable params: 1,843,904 (7.03 MB)

ResNet50

In [10]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model

def upsample(filters, size):
    """Bloc d’upsampling"""
    return tf.keras.Sequential([
        layers.Conv2DTranspose(filters, size, strides=2, padding="same"),
        layers.BatchNormalization(),
        layers.ReLU()
    ])

def create_resnet50_unet(input_shape=(224,224,3)):
    # Charge ResNet50 comme backbone
    base_model = ResNet50(input_shape=input_shape, include_top=False, weights="imagenet")

    # Skip connections : on choisit des features à différentes résolutions
    layer_names = [
        "conv1_relu",   # 112x112
        "conv2_block3_out",  # 56x56
        "conv3_block4_out",  # 28x28
        "conv4_block6_out",  # 14x14
        "conv5_block3_out",  # 7x7
    ]
    layers_outputs = [base_model.get_layer(name).output for name in layer_names]

    encoder = Model(inputs=base_model.input, outputs=layers_outputs)
    encoder.trainable = False  # freeze au début

    # Input
    inputs = layers.Input(shape=input_shape)
    skips = encoder(inputs)
    x = skips[-1]  # sortie la plus profonde (7x7)

    # Décodeur avec skip connections
    up_stack = [
        upsample(512, 3),  # 14x14
        upsample(256, 3),  # 28x28
        upsample(128, 3),  # 56x56
        upsample(64, 3),   # 112x112
    ]

    for up, skip in zip(up_stack, reversed(skips[:-1])):
        x = up(x)
        x = layers.Concatenate()([x, skip])

    # Dernière couche (retour taille originale)
    last = layers.Conv2DTranspose(1, 3, strides=2, padding="same", activation="sigmoid")
    outputs = last(x)

    return Model(inputs, outputs)

# Création du modèle
model = create_resnet50_unet()
model.summary()


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ [(None, 112, 112, │ 23,587,712 │ input_layer_1[0]… │
│ (Functional)        │ 64), (None, 56,   │            │                   │
│                     │ 56, 256), (None,  │            │                   │
│                     │ 28, 28, 512),     │            │                   │
│                     │ (None, 14, 14,    │            │                   │
│                     │ 1024), (None, 7,  │            │                   │
│                     │ 7, 2048)]         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 14, 14,    │  9,439,744 │ functional[0][4]  │
│ (Sequential)        │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 14, 14,    │          0 │ sequential[0][0], │
│ (Concatenate)       │ 1536)             │            │ functional[0][3]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 28, 28,    │  3,540,224 │ concatenate[0][0] │
│ (Sequential)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 28, 28,    │          0 │ sequential_1[0][… │
│ (Concatenate)       │ 768)              │            │ functional[0][2]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, 56, 56,    │    885,376 │ concatenate_1[0]… │
│ (Sequential)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 56, 56,    │          0 │ sequential_2[0][… │
│ (Concatenate)       │ 384)              │            │ functional[0][1]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_3        │ (None, 112, 112,  │    221,504 │ concatenate_2[0]… │
│ (Sequential)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 112, 112,  │          0 │ sequential_3[0][… │
│ (Concatenate)       │ 128)              │            │ functional[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_4  │ (None, 224, 224,  │      1,153 │ concatenate_3[0]… │
│ (Conv2DTranspose)   │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 37,675,713 (143.72 MB)

 Trainable params: 14,086,081 (53.73 MB)

 Non-trainable params: 23,589,632 (89.99 MB)

InceptionV3

In [51]:
# ------------------------------------------------------------
# U-Net avec InceptionV3 (encodeur) pour segmentation binaire
# Input  : 224x224x3
# Output : 224x224x1 (sigmoid)
# ------------------------------------------------------------
import tensorflow as tf
from tensorflow.keras import layers as L, models as M, backend as K
from tensorflow.keras.applications import InceptionV3

# --------- Loss & Metrics ----------
def dice_coefficient(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    denom = K.sum(y_true_f) + K.sum(y_pred_f)
    return (2. * intersection + smooth) / (denom + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return bce + dice_loss(y_true, y_pred)

# --------- Blocks ----------
def conv_block(x, filters, kernel_size=3, bn=True):
    x = L.Conv2D(filters, kernel_size, padding="same")(x)
    if bn: x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
    x = L.Conv2D(filters, kernel_size, padding="same")(x)
    if bn: x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
    return x

def resize_to(skip, x):
    target_h, target_w = x.shape[1], x.shape[2]
    return L.Resizing(target_h, target_w, interpolation="bilinear")(skip)

def up_block(x, skip, filters):
    x = L.Conv2DTranspose(filters, kernel_size=2, strides=2, padding="same")(x)
    if skip is not None:
        skip = L.Conv2D(filters, 1, padding="same")(skip)
        skip = resize_to(skip, x)   # 🔑 redimensionnement ici
        x = L.Concatenate()([x, skip])
    x = conv_block(x, filters)
    return x

# --------- Model ----------
def build_unet_inceptionv3(input_shape=(224, 224, 3), freeze_encoder=True):
    # InceptionV3 comme encodeur (pré-entraîné ImageNet)
    base = InceptionV3(include_top=False, weights="imagenet", input_shape=input_shape)

    # On récupère des “skip connections” robustes par nom (stables: mixed0..mixed10)
    # Tailles spatiales attendues pour input 224x224:
    #   mixed2  ~ 28x28
    #   mixed5  ~ 14x14
    #   mixed10 ~ 7x7  (bottleneck)
    skip_56 = base.get_layer("mixed0").output
    skip_28 = base.get_layer("mixed3").output
    skip_14 = base.get_layer("mixed6").output
    bottleneck = base.get_layer("mixed10").output

    if freeze_encoder:
        for l in base.layers:
            l.trainable = False

    # Decoder U-Net (7 -> 14 -> 28 -> 56 -> 112 -> 224)
    x = bottleneck
    x = up_block(x, skip_14, 512)     # 7 -> 14
    x = up_block(x, skip_28, 256)     # 14 -> 28
    x = up_block(x, None, 128)        # 28 -> 56
    x = up_block(x, None, 64)         # 56 -> 112
    x = up_block(x, None, 32)         # 112 -> 224

    # Sortie binaire (1 canal), même HxW que l'entrée
    #outputs = L.Conv2D(1, kernel_size=1, activation="sigmoid", padding="same")(x)
    outputs = L.Conv2D(1, kernel_size=1, activation="sigmoid", padding="same")(x)
    outputs = L.Resizing(224, 224, interpolation="bilinear")(outputs)  # 🔑 ajout

    model = M.Model(inputs=base.input, outputs=outputs, name="UNet_InceptionV3_224")
    return model

model = build_unet_inceptionv3(input_shape=(224, 224, 3), freeze_encoder=True)
model.summary()
    # Construction du modèle


    # EXEMPLE de pipeline d’entraînement (placeholders)
    # Remplace X_train, Y_train, X_val, Y_val par tes tenseurs/TFRecords/TF Datasets.
    # ATTENTION: Y_* doivent être des masques binaires shape (224,224,1) avec valeurs 0/1.
    # Prétraitement conseillé: normaliser X_* en [0,1] et, si possible, augmenter (flips, rotations légères).

    # Exemple minimal avec tf.data (adapte à tes données):
    # import numpy as np
    # X_train = np.random.rand(8, 224, 224, 3).astype("float32")
    # Y_train = (np.random.rand(8, 224, 224, 1) > 0.5).astype("float32")
    # train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(4).prefetch(tf.data.AUTOTUNE)

    # history = model.fit(train_ds, epochs=20, validation_data=train_ds)

    # --------- Phase de fine-tuning (optionnelle) ----------
    # Dégriser certaines couches “Inception” pour adapter davantage au domaine des fissures.
    # Débloque par exemple à partir de 'mixed5' (gèle les couches en amont).
    # unfreeze_from = "mixed5"
    # unfreeze = False
    # for l in model.layers:
        # if l.name == unfreeze_from:
        #     unfreeze = True
        # if unfreeze and l.name in [ly.name for ly in model.get_layer("inception_v3").layers]:
        #     l.trainable = True
    # model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
    #               loss=bce_dice_loss,
    #               metrics=[dice_coefficient, "accuracy"])
    # history_ft = model.fit(train_ds, epochs=10, validation_data=train_ds)


Model: "UNet_InceptionV3_224"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_41      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1847         │ (None, 111, 111,  │        864 │ input_layer_41[0… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d_1847[0][0] │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1796     │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1848         │ (None, 109, 109,  │      9,216 │ activation_1796[… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1848[0][0] │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1797     │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1849         │ (None, 109, 109,  │     18,432 │ activation_1797[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_1849[0][0] │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1798     │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_76    │ (None, 54, 54,    │          0 │ activation_1798[… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1850         │ (None, 54, 54,    │      5,120 │ max_pooling2d_76… │
│ (Conv2D)            │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_1850[0][0] │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1799     │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1851         │ (None, 52, 52,    │    138,240 │ activation_1799[… │
│ (Conv2D)            │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_1851[0][0] │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1800     │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 36,529,377 (139.35 MB)

 Trainable params: 14,722,625 (56.16 MB)

 Non-trainable params: 21,806,752 (83.19 MB)

EfficientNetB7

In [55]:
import tensorflow as tf
from tensorflow.keras import layers as L, models as M
from tensorflow.keras.applications import EfficientNetB7

# --------- Blocks ----------
def conv_block(x, filters, kernel_size=3, bn=True):
    x = L.Conv2D(filters, kernel_size, padding="same")(x)
    if bn: x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
    x = L.Conv2D(filters, kernel_size, padding="same")(x)
    if bn: x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
    return x

def resize_to(skip, x):
    """resize skip tensor to match x spatial dims"""
    return L.Resizing(x.shape[1], x.shape[2], interpolation="bilinear")(skip)

def up_block(x, skip, filters):
    x = L.Conv2DTranspose(filters, kernel_size=2, strides=2, padding="same")(x)
    if skip is not None:
        skip = L.Conv2D(filters, 1, padding="same")(skip)
        skip = resize_to(skip, x)
        x = L.Concatenate()([x, skip])
    x = conv_block(x, filters)
    return x

# --------- Model ----------
def build_unet_efficientnetb7(input_shape=(224,224,3), freeze_encoder=True):
    base = EfficientNetB7(include_top=False, weights="imagenet", input_shape=input_shape)

    # Skip connections (repères connus dans EfficientNetB7)
    skips = [
        base.get_layer("block2a_expand_activation").output,  # ~56x56
        base.get_layer("block3a_expand_activation").output,  # ~28x28
        base.get_layer("block4a_expand_activation").output,  # ~14x14
        base.get_layer("block6a_expand_activation").output,  # ~7x7
    ]
    bottleneck = base.get_layer("top_activation").output  # ~7x7

    if freeze_encoder:
        for l in base.layers:
            l.trainable = False

    # Decoder
    x = bottleneck
    x = up_block(x, skips[-1], 512)  # 7 -> 14
    x = up_block(x, skips[-2], 256)  # 14 -> 28
    x = up_block(x, skips[-3], 128)  # 28 -> 56
    x = up_block(x, skips[-4], 64)   # 56 -> 112
    x = up_block(x, None, 32)        # 112 -> 224

    outputs = L.Conv2D(1, 1, activation="sigmoid", padding="same")(x)

    # 🔑 corrige la sortie pour matcher l’input
    outputs = L.Resizing(input_shape[0], input_shape[1])(outputs)

    model = M.Model(inputs=base.input, outputs=outputs, name="UNet_EfficientNetB7")
    return model


model = build_unet_efficientnetb7(input_shape=(224,224,3))
model.summary()


258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "UNet_EfficientNetB7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_42      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer_42[0… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │      1,728 │ stem_conv_pad[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        256 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        576 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        256 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 64)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 64)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 16)  │      1,040 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 64)  │      1,088 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 64)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │      2,048 │ block1a_se_excit

 Total params: 80,327,704 (306.43 MB)

 Trainable params: 16,226,049 (61.90 MB)

 Non-trainable params: 64,101,655 (244.53 MB)

Novel

In [62]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy","recall","precision"]
)


MobileNetV2

In [ ]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=8,
    epochs=16
)


Epoch 1/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 223s 1s/step - accuracy: 0.8824 - loss: 0.3829 - precision: 0.0458 - recall: 0.1783 - val_accuracy: 0.9759 - val_loss: 0.1163 - val_precision: 0.9262 - val_recall: 0.0070
Epoch 2/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.9797 - loss: 0.0947 - precision: 0.6731 - recall: 0.0877 - val_accuracy: 0.9775 - val_loss: 0.0814 - val_precision: 0.9021 - val_recall: 0.0838
Epoch 3/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 213s 1s/step - accuracy: 0.9812 - loss: 0.0686 - precision: 0.7136 - recall: 0.1882 - val_accuracy: 0.9801 - val_loss: 0.0689 - val_precision: 0.8530 - val_recall: 0.2157
Epoch 4/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.9830 - loss: 0.0579 - precision: 0.7186 - recall: 0.2924 - val_accuracy: 0.9815 - val_loss: 0.0627 - val_precision: 0.8276 - val_recall: 0.3021
Epoch 5/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 210s 1s/step - accuracy: 0.9834 - loss: 0.0543 - precision: 0.7271 - recall: 0.3464 - val_accuracy: 0.9826 - val_los

In [ ]:
model.save("unet2_segmentation_fissures_mobilenetv2.keras")

ResNet50

In [12]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=8,
    epochs=16
)


Epoch 1/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 495s 3s/step - accuracy: 0.7874 - loss: 0.4092 - precision: 0.0221 - recall: 0.2009 - val_accuracy: 0.9769 - val_loss: 0.1473 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 420s 2s/step - accuracy: 0.9789 - loss: 0.1281 - precision: 0.1163 - recall: 9.1706e-05 - val_accuracy: 0.9769 - val_loss: 0.1148 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 490s 3s/step - accuracy: 0.9781 - loss: 0.0951 - precision: 0.5718 - recall: 0.0242 - val_accuracy: 0.9769 - val_loss: 0.0975 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 453s 2s/step - accuracy: 0.9798 - loss: 0.0786 - precision: 0.6241 - recall: 0.0760 - val_accuracy: 0.9774 - val_loss: 0.0804 - val_precision: 0.9601 - val_recall: 0.0218
Epoch 5/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 497s 2s/step - accuracy: 0.9799 - loss: 0.0732 - precision: 0.6216 - recall: 0.1246 - va

In [13]:
model.save("unet_resnet50_segmentation_fissures_mobilenetv2.keras")

InceptionV3

In [53]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=8,
    epochs=16
)


Epoch 1/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 510s 3s/step - accuracy: 0.9094 - loss: 0.4041 - precision: 0.0375 - recall: 0.1180 - val_accuracy: 0.9754 - val_loss: 0.3697 - val_precision: 0.1083 - val_recall: 0.0086
Epoch 2/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 492s 3s/step - accuracy: 0.9780 - loss: 0.1823 - precision: 0.1281 - recall: 0.0052 - val_accuracy: 0.9748 - val_loss: 0.1810 - val_precision: 0.1416 - val_recall: 0.0180
Epoch 3/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 487s 3s/step - accuracy: 0.9786 - loss: 0.1473 - precision: 0.2632 - recall: 0.0108 - val_accuracy: 0.9762 - val_loss: 0.1356 - val_precision: 0.2689 - val_recall: 0.0160
Epoch 4/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 488s 3s/step - accuracy: 0.9795 - loss: 0.1256 - precision: 0.5901 - recall: 0.0465 - val_accuracy: 0.9778 - val_loss: 0.1163 - val_precision: 0.6908 - val_recall: 0.0714
Epoch 5/16
188/188 ━━━━━━━━━━━━━━━━━━━━ 488s 3s/step - accuracy: 0.9801 - loss: 0.1117 - precision: 0.6536 - recall: 0.1136 - val_accuracy: 0.9784 - val_los

In [54]:
model.save("unet_inceptionv3_segmentation_fissures_mobilenetv2.keras")

EfficientNetB7

In [ ]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=8,
    epochs=16
)


Epoch 1/16
  3/188 ━━━━━━━━━━━━━━━━━━━━ 1:42:18 33s/step - accuracy: 0.0970 - loss: 1.0785 - precision: 0.0231 - recall: 0.9567

In [ ]:
model.save("unet_EfficientNetB7_segmentation_fissures_mobilenetv2.keras")

END

In [ ]:
!ls

data				      segmentation_fissures_mobilenetv2.keras
segmentation_fissures_mobilenetv2.h5  valid_mask.zip


In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras
